In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
spacex_df = pd.read_csv(URL)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [12]:
# Add circles + markers
for _, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']

    # Circle
    folium.Circle(
        coordinate,
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(launch_site_name)).add_to(site_map)

    # Marker (text label)
    folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{launch_site_name}</b></div>'
        )
    ).add_to(site_map)

site_map

In [13]:
# Cluster para markers
marker_cluster = MarkerCluster().add_to(site_map)

In [14]:
# Función de color
def marker_color(launch_class):
    return 'green' if launch_class == 1 else 'red'

In [15]:
# Agregamos un marker por lanzamiento (success/failed)
for _, row in spacex_df.dropna(subset=['Lat', 'Long']).iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site = row.get('Launch Site', 'Unknown')
    flight_no = row.get('FlightNumber', row.get('flight_number', ''))
    launch_class = int(row['class']) if str(row['class']).isdigit() else row['class']

    popup_text = f"{launch_site}<br>Flight: {flight_no}<br>Class: {launch_class}"

    folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color(launch_class), icon='info-sign'),
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(marker_cluster)

site_map

In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [21]:
# 1) Coordenadas del launch site (ej: CCAFS LC-40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# 2) Coordenadas del punto más cercano de costa (las sacás con MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# 3) Distancia en km
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon,
    coastline_lat, coastline_lon
)

distance_coastline

0.5797120571581802

In [22]:
# Coordenadas
launch_coordinate = [launch_site_lat, launch_site_lon]
coastline_coordinate = [coastline_lat, coastline_lon]

# Distancia (km)
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon,
    coastline_lat, coastline_lon
)

# Marker en el punto de costa
folium.Marker(
    coastline_coordinate,
    icon=folium.Icon(color='blue', icon='info-sign'),
    popup='Closest coastline point'
).add_to(site_map)

# Línea entre launch site y costa
folium.PolyLine(
    locations=[launch_coordinate, coastline_coordinate],
    weight=2
).add_to(site_map)

# Label con la distancia (texto en el mapa)
distance_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' 
             % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(distance_marker)

site_map

In [23]:
# Launch site coordinate (ej: CCAFS LC-40)
launch_coordinate = [launch_site_lat, launch_site_lon]

def add_proximity(site_map, launch_coord, target_name, target_lat, target_lon):
    # distance in KM
    dist = calculate_distance(launch_coord[0], launch_coord[1], target_lat, target_lon)
    target_coord = [target_lat, target_lon]

    # point marker
    folium.Marker(
        target_coord,
        popup=f"{target_name} ({dist:.2f} km)",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

    # line from launch site to target
    folium.PolyLine(
        locations=[launch_coord, target_coord],
        weight=2
    ).add_to(site_map)

    # distance label (DivIcon)
    folium.Marker(
        target_coord,
        icon=DivIcon(
            icon_size=(160, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{dist:.2f} KM</b></div>'
        )
    ).add_to(site_map)

# ---- Reemplazá estas coords con las que tomás del MousePosition ----
# Ejemplos (NO son universales): poné las tuyas reales cerca del launch site

# Closest coastline point
coast_lat, coast_lon = 28.56367, -80.56774

# Closest highway point (poné coords reales)
highway_lat, highway_lon = 28.56311, -80.57084

# Closest railway point (poné coords reales)
rail_lat, rail_lon = 28.57337, -80.65399

# Add proximities
add_proximity(site_map, launch_coordinate, "Closest Coastline", coast_lat, coast_lon)
add_proximity(site_map, launch_coordinate, "Closest Highway", highway_lat, highway_lon)
add_proximity(site_map, launch_coordinate, "Closest Railway", rail_lat, rail_lon)

site_map